In [1]:
import pytds
import re 
import spacy
import pandas as pd
from time import time
connect_string = ""
try:
    conn = pytds.connect(server = '2605:ad80:30:5058:84ea:3de3:39a1:f11', database = 'TBDraft', user = 'admin1', password='TBuilder')
    print("{c} is working".format(c=connect_string))
except pytds.Error as ex:
    print("{c} is not working".format(c=connect_string))

 is working


In [2]:
from tika import parser
import os
df = pd.DataFrame(columns=['CandidateID','Keyword', 'Resume', 'Cleaned_Resume'])
for dirname , _, filenames in os.walk('C:/Users/maxbr/ECEN 403/ECEN-403/Train Resumes'):
    for filename in filenames:
       path = os.path.join(dirname, filename)
       raw = parser.from_file(path)
       df = df.append({'CandidateID' : filename[:-4],'Keyword' : dirname.replace("C:/Users/maxbr/ECEN 403/ECEN-403/Train Resumes\\", ""), 'Resume' : " ".join(raw['content'].strip().split('\n')[1:]), 'Cleaned_Resume' : ""}, 
                ignore_index = True)
   
df.shape

def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText
df['Cleaned_Resume'] = df.Resume.apply(lambda x: cleanResume(x))
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['Cleaned_Resume'])

t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))
df['Cleaned_Resume'] = txt
df.head

2021-11-29 17:52:04,363 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Time to clean up everything: 0.15 mins


<bound method NDFrame.head of               CandidateID               Keyword  \
0         Daniel-Whatcott   Biomedical_Engineer   
1          Patrick-Hannah   Biomedical_Engineer   
2           Abraham-Ajayi  Electronics_Engineer   
3          Adam-Blakeslee  Electronics_Engineer   
4               Alaa-Issa  Electronics_Engineer   
..                    ...                   ...   
144         Nathan-Nguyen     Software_Engineer   
145  Omar-Trueba-Mijangos     Software_Engineer   
146          Peter-Pagano     Software_Engineer   
147        Steven-Nichols     Software_Engineer   
148          Xueqiang-Pan     Software_Engineer   

                                                Resume  \
0      Daniel Whatcott Service Manager at BIOMEDICA...   
1      Patrick Hannah Electrical Engineer II - Vare...   
2      Abraham Ajayi Northridge, CA -Email me on In...   
3      Adam Blakeslee North Ogden, UT -Email me on ...   
4      Alaa Issa Electrical Engineering Iraq- Sulay...   
..       

In [3]:
df

,CandidateID,Keyword,Resume,Cleaned_Resume
0,Daniel-Whatcott,Biomedical_Engineer,Daniel Whatcott Service Manager at BIOMEDICA...,daniel whatcott service manager biomedical e...
1,Patrick-Hannah,Biomedical_Engineer,Patrick Hannah Electrical Engineer II - Vare...,patrick hannah electrical engineer ii varex ...
2,Abraham-Ajayi,Electronics_Engineer,"Abraham Ajayi Northridge, CA -Email me on In...",abraham ajayi northridge email seek challeng...
3,Adam-Blakeslee,Electronics_Engineer,"Adam Blakeslee North Ogden, UT -Email me on ...",adam blakeslee north ogden ut email work exp...
4,Alaa-Issa,Electronics_Engineer,Alaa Issa Electrical Engineering Iraq- Sulay...,alaa issa electrical engineering iraq sulaym...
...,...,...,...,...
144,Nathan-Nguyen,Software_Engineer,"Nathan Nguyen Irvine, CA -Email me on Indeed...",nathan nguyen irvine email work experience s...
145,Omar-Trueba-Mijangos,Software_Engineer,Omar Trueba Mijangos Computer Engineer San J...,omar trueba mijango computer engineer san jo...
146,Peter-Pagano,Software_Engineer,"Peter Pagano Bohemia, NY -Email me on Indeed...",peter pagano bohemia ny email authorize work...
147,Steven-Nichols,Software_Engineer,Steven Nichols C/C++ Software Engineer Grand...,steven nichol c c software engineer grand ra...


In [4]:
cursor = conn.cursor()
cursor.execute('''TRUNCATE TABLE TrainingResumes''')
conn.commit()
#cursor.execute("INSERT INTO HumanResources.DepartmentTest (DepartmentID,Name,GroupName) values(?,?,?)", row.DepartmentID, row.Name, row.GroupName)
insert_q = "INSERT INTO TrainingResumes (keyword, dataText) "
for i in range(len(df['Cleaned_Resume'])):
    keyword = df.Keyword[i]
    data = df.Cleaned_Resume[i]

    values = "Values('" + keyword + "','" + data + "')"
    commd = insert_q + values
    
    cursor.execute(commd)
    
conn.commit()

In [6]:
df_training = pd.read_sql_query("SELECT ALL * FROM TrainingResumes", conn)
conn.close()
df_training

Error: Connection closed

In [8]:
import pytds
connect_string = ""
try:
    conn = pytds.connect(server = '34.69.166.47', database = 'TBDraft', user = 'sqlserver', password='TBuilder')
    print("{c} is working".format(c=connect_string))
except pytds.Error as ex:
    print("{c} is not working".format(c=connect_string))

 is not working


In [31]:
df_roles = pd.read_sql_query("SELECT ALL * FROM Roles", conn)
df_projects = pd.read_sql_query("SELECT ALL * FROM Project", conn)

display(df_projects)
display(df_roles)

,projectName,Descrp,ProfessorName


,projectName,roleName,roleDescription,conskills,proskills,Candidate1,Candidate2,Candidate3
0,Solar Skateboard,Role 1,Power Supply Design,,"Power Supply, DC Converter",0,0,0
1,Solar Skateboard,Role 2,Microcontroller,,"Python, C, Embedded Systems",0,0,0
2,Solar Skateboard,Role 3,Microcontroller,Java,"Wireless Controller, RF",0,0,0
3,Music Project,Role 1,App Developer,,"Python, Coding, Java",0,0,0
4,Music Project,Role 2,Microcontroller/Bluetooth,Web Development,"Microcontroller, Wireless, Bluetooth",0,0,0
5,Music Project,Role 3,Audio Transmission,Power Supply Design,"ADC, Audio Transmission",0,0,0
6,Music Project,Role 4,Power Supply Design,,"Power Supply, Converter, Transformer",0,0,0


In [30]:
cursor = conn.cursor()
insert_q = "INSERT INTO Roles (projectName, roleName, roleDescription, Candidate1, Candidate2, Candidate3, proSkills, conSkills ) "
values = "Values('Music Project', 'Role 4', 'Power Supply Design', '' , '' , '', 'Power Supply, Converter, Transformer', '')"
commd = insert_q + values   
cursor.execute(commd)    
conn.commit()

In [14]:
# cursor.execute('''TRUNCATE TABLE Roles''')
# conn.commit()

In [37]:
conn.close()